In [1]:
import pandas as pd
from pathlib import Path
import os
from Step_4_MLmodels.PrepareDatasetForLearning import PrepareDatasetForLearning
from Step_4_MLmodels.LearningAlgorithms import ClassificationAlgorithms
from Step_4_MLmodels.Evaluation import ClassificationEvaluation
from Step_4_MLmodels.FeatureSelection import FeatureSelectionClassification
from util import util
from util.VisualizeDataset import VisualizeDataset
from sklearn.ensemble import RandomForestClassifier

# Set up file names and locations.
FOLDER_PATH = Path('C:\\Users\\DUC_AN\\Documents\\GitHub\\EEG-ICSSE\\intermediate_datafiles\\motor_imagery\\step3_result\\all')
RESULT_PATH = Path('C:\\Users\\DUC_AN\\Documents\\GitHub\\EEG-ICSSE\\intermediate_datafiles\\motor_imagery\\step4_result\\all')

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
import pandas as pd
import numpy as np
import os
import pickle
import inspect

In [3]:
RESULT_PATH.mkdir(exist_ok=True, parents=True)
# for this script, we want to first load in all datasets
# since the Prepare dataset function accepts a list of pd dataframes
prepare = PrepareDatasetForLearning()
all_datasets = []

for instance in os.scandir(FOLDER_PATH): # go through all instances of experiments
    instance_path = instance.path
    dataset = pd.read_csv(instance_path, index_col=0)
    dataset.index = pd.to_datetime(dataset.index)
    all_datasets.append(dataset)

#now all dataframes are added to the list all_datasets
#print(all_datasets)

# Let's create our visualization class again.

'''
the classification of the motor imagery can be seen as a non-temporal task, as we want to predict imagery based on a window of e.g. 2 sec,
without taking into account previous windows.
We first create 1 column representing our classes, and then create a train val test split of 60 20 20
In order to do this, we first create a train test split of 80 20, and then for the train set we split again in 75 25
For each dataset instance. we split trainvaltest split individually.
Then later we add all train data together, all val data together, and all test data together.
This way we sample randomly across all users to get a result for the whole 'population' of subjects.
'''
# we set filter is false so also the data besides left and right are taken with us
train_X, val_X, test_X, train_y, val_y, test_y = prepare.split_multiple_datasets_classification(
    all_datasets, ['0', '1'], 'like', [0.2, 0.25],filter=False, temporal=False)
print('Training set length is: ', len(train_X.index))
print('Validation set length is: ', len(val_X.index))
print('Test set length is: ', len(test_X.index))   

# select subsets of features which we will consider:
pca_features = ['pca_1','pca_2','pca_3','pca_4']
ica_features = ['FastICA_1','FastICA_2','FastICA_3','FastICA_4','FastICA_5','FastICA_6','FastICA_7','FastICA_8','FastICA_9','FastICA_10',
'FastICA_11','FastICA_12','FastICA_13','FastICA_14','FastICA_15','FastICA_16','FastICA_17','FastICA_18','FastICA_19','FastICA_20']
time_features = [name for name in dataset.columns if '_temp_' in name]
freq_features = [name for name in dataset.columns if (('_freq' in name) or ('_pse' in name))]


# feature selection below we will use as input for our models:
basic_features = ['Delta_TP9','Delta_AF7','Delta_AF8','Delta_TP10','Theta_TP9','Theta_AF7','Theta_AF8','Theta_TP10','Alpha_TP9','Alpha_AF7',
'Alpha_AF8','Alpha_TP10','Beta_TP9','Beta_AF7','Beta_AF8','Beta_TP10','Gamma_TP9','Gamma_AF7','Gamma_AF8','Gamma_TP10']
basic_w_PCA = list(set().union(basic_features, pca_features))
basic_w_ICA = list(set().union(basic_features, ica_features))
all_features = list(set().union(basic_features, ica_features, time_features, freq_features))

fs = FeatureSelectionClassification()
num_features = 20

# we will select the top 20 features based on an experiment with a deciscion tree which we will use as input for our models as well
# this is already been run, see below

'''
selected_features, ordered_features, ordered_scores = fs.forward_selection(num_features,
                                                              train_X[all_features],
                                                              test_X[all_features],
                                                              train_y,
                                                              test_y,
                                                              gridsearch=False)
print(selected_features)
'''

# the best feature are right now:
selected_features = ['Delta_AF7_temp_max_ws_10', 'Alpha_TP9_temp_mean_ws_10', 'Delta_AF7_temp_slope_ws_30', 'FastICA_2', 
'Alpha_TP9_temp_median_ws_20', 'Delta_AF8_temp_max_ws_10', 'Beta_TP10_freq_30.0_Hz_ws_10', 
'Beta_TP9_temp_std_ws_20', 'Theta_TP10_temp_max_ws_20', 'Gamma_TP9_temp_median_ws_20',
 'Gamma_TP10_freq_30.0_Hz_ws_10', 'Alpha_TP10_temp_std_ws_20', 'Gamma_AF7_freq_30.0_Hz_ws_10', 'Delta_TP10', 
 'Beta_TP9_temp_median_ws_20', 'Delta_TP10_temp_min_ws_20', 'Theta_TP9_temp_median_ws_30', 
 'Delta_AF8_temp_min_ws_20', 'Delta_AF8_temp_mean_ws_10', 'Beta_TP9_freq_0.0_Hz_ws_10']


possible_feature_sets = [basic_features, basic_w_PCA, basic_w_ICA, all_features, selected_features]
feature_names = ['initial set', 'basic_w_PCA', 'basic_w_ICA', 'all_features', 'Selected features']
N_KCV_REPEATS = 10 # some non deterministic models we will run a couple of times as their inits are random to get average results


# then here, we run each model
learner = ClassificationAlgorithms()
eval = ClassificationEvaluation()
scores_over_all_algs = []

1900-01-01 00:21:15.800    0.0
1900-01-01 00:21:17.300    0.0
1900-01-01 00:21:18.800    0.0
1900-01-01 00:21:20.300    0.0
1900-01-01 00:21:21.800    0.0
                          ... 
1900-01-01 00:26:18.800    0.0
1900-01-01 00:26:20.300    0.0
1900-01-01 00:26:21.800    0.0
1900-01-01 00:26:23.300    0.0
1900-01-01 00:26:24.800    0.0
Length: 207, dtype: float64
1900-01-01 00:52:38.000    0.0
1900-01-01 00:52:39.500    0.0
1900-01-01 00:52:41.000    0.0
1900-01-01 00:52:42.500    0.0
1900-01-01 00:52:44.000    0.0
                          ... 
1900-01-01 00:57:45.500    0.0
1900-01-01 00:57:47.000    0.0
1900-01-01 00:57:48.500    0.0
1900-01-01 00:57:50.000    0.0
1900-01-01 00:57:51.500    0.0
Length: 210, dtype: float64
1900-01-01 00:03:32.000    0.0
1900-01-01 00:03:33.500    0.0
1900-01-01 00:03:35.000    0.0
1900-01-01 00:03:36.500    0.0
1900-01-01 00:03:38.000    0.0
                          ... 
1900-01-01 00:08:41.000    0.0
1900-01-01 00:08:42.500    0.0
1900-01-01 00:

In [4]:
train_y.nunique()

check    2
dtype: int64

In [5]:
train_y

,check
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1900-01-01 00:27:16.700000,0.0
1900-01-01 00:25:19.700000,0.0
1900-01-01 00:24:57.200000,0.0
1900-01-01 00:27:31.700000,0.0


In [6]:
print(len(train_X))

3132


In [7]:
# Apply a random forest approach for classification upon the training data (with the specified value for
# the minimum samples in the leaf, the number of trees, and if we should print some of the details of the
# model print_model_details=True) and use the created model to predict the outcome for both the
# test and training set. It returns the categorical predictions for the training and test set as well as the
# probabilities associated with each class, each class being represented as a column in the data frame.
def random_forest(train_X, train_y, test_X, n_estimators=10, min_samples_leaf=5, criterion='gini', print_model_details=False, gridsearch=True, save_model=False):

    if gridsearch:
        tuned_parameters = [{'min_samples_leaf': [2, 10, 50, 100, 200],
                                'n_estimators':[10, 50, 100],
                                'criterion':['gini', 'entropy']}]
        rf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5, scoring='accuracy', error_score= 'raise')
    else:
        rf = RandomForestClassifier(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, criterion=criterion)

    # Fit the model

    rf.fit(train_X, train_y.check.ravel())

    if gridsearch and print_model_details:
        print(rf.best_params_)

    if gridsearch:
        rf = rf.best_estimator_

    pred_prob_training_y = rf.predict_proba(train_X)
    pred_prob_test_y = rf.predict_proba(test_X)
    pred_training_y = rf.predict(train_X)
    pred_test_y = rf.predict(test_X)
    frame_prob_training_y = pd.DataFrame(pred_prob_training_y, columns=rf.classes_)
    frame_prob_test_y = pd.DataFrame(pred_prob_test_y, columns=rf.classes_)

    if print_model_details:
        ordered_indices = [i[0] for i in sorted(enumerate(rf.feature_importances_), key=lambda x:x[1], reverse=True)]
        print('Top 20 feature importances random forest:')
        for i in range(0, 20):
            print(train_X.columns[ordered_indices[i]], end='')
            print(' & ', end='')
            print(rf.feature_importances_[ordered_indices[i]])
    
    if save_model:
        # save the model to disk
        filename = 'final_' + str(inspect.stack()[0][3]) + '_model_BCI.sav'
        pickle.dump(rf, open(filename, 'wb'))

    return pred_training_y, pred_test_y, frame_prob_training_y, frame_prob_test_y

In [8]:
train_y.check.unique()

array([1., 0.])

In [9]:
train_X.nunique()

Delta_TP9                        2266
Delta_AF7                        2513
Delta_AF8                        2533
Delta_TP10                       2210
Theta_TP9                        2266
                                 ... 
Gamma_TP10_freq_20.0_Hz_ws_10    1993
Gamma_TP10_freq_30.0_Hz_ws_10    1996
Gamma_TP10_freq_40.0_Hz_ws_10    1998
Gamma_TP10_freq_50.0_Hz_ws_10    2035
class                               1
Length: 585, dtype: int64

In [10]:
train_X.fillna(0)

,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,Alpha_AF7,...,Gamma_TP10_max_freq_ws_10,Gamma_TP10_freq_weighted_ws_10,Gamma_TP10_pse_ws_10,Gamma_TP10_freq_0.0_Hz_ws_10,Gamma_TP10_freq_10.0_Hz_ws_10,Gamma_TP10_freq_20.0_Hz_ws_10,Gamma_TP10_freq_30.0_Hz_ws_10,Gamma_TP10_freq_40.0_Hz_ws_10,Gamma_TP10_freq_50.0_Hz_ws_10,class
0,0.624402,0.717011,0.919645,0.825707,0.434732,0.115024,0.410620,0.720329,0.648855,-0.082225,...,3.906546e-01,-9.518690e+00,0.0,0.471205,-1.672656e-01,-3.406519e-02,-1.800379e-02,9.280505e-03,6.223235e-04,0.0
1,0.587482,0.788116,0.932912,0.532051,0.385909,0.638301,0.883139,0.409161,0.734609,0.610241,...,4.598610e-02,3.279093e+00,20.0,-0.943916,-6.489695e-02,1.018178e-03,-2.257550e-02,-2.366062e-02,-2.576942e-02,0.0
2,1.123026,0.859138,0.532642,1.219550,1.124511,0.327637,0.032039,1.094191,1.066517,0.389382,...,4.251746e-01,-5.265867e+01,0.0,0.761696,-1.292887e-01,-1.470459e-01,-7.400607e-02,-5.530642e-02,-9.821715e-02,0.0
3,0.657415,0.306352,0.768359,0.516933,0.541668,0.247029,0.336359,0.418441,0.552087,0.595063,...,1.565297e+00,4.080327e+01,20.0,-0.020256,-1.825677e-02,5.910165e-02,5.320403e-02,5.021667e-02,4.955530e-02,0.0
4,0.552601,0.338759,0.641789,0.529005,0.139140,-0.474317,-0.358568,0.081230,0.636873,0.053596,...,5.814059e-01,1.406718e+01,0.0,0.443273,6.152076e-02,7.756041e-02,8.382619e-02,7.957028e-02,7.309734e-02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900-01-01 00:27:16.700000,1.630634,0.849937,0.805977,1.186253,1.031788,0.672794,0.377008,0.837470,0.848964,0.405224,...,6.735395e-03,1.636710e+00,50.0,-1.950003,-2.344050e-02,-2.312702e-02,-2.271643e-02,-2.233911e-02,-2.211485e-02,0.0
1900-01-01 00:25:19.700000,0.585597,1.287282,0.756415,0.601682,0.201669,0.867307,0.608366,-0.054366,0.231963,0.907556,...,2.571454e-31,3.801630e-15,30.0,-0.985630,-2.775558e-17,-2.775558e-17,-1.387779e-17,-2.775558e-17,-2.775558e-17,0.0
1900-01-01 00:24:57.200000,0.695946,0.501303,0.603439,0.785471,0.582576,0.194988,0.082204,0.215667,0.575390,0.397730,...,1.900079e-02,9.915513e-01,10.0,-1.441876,6.128236e-02,2.742406e-03,-3.178164e-02,-1.113937e-02,-1.382244e-02,0.0
1900-01-01 00:27:31.700000,0.356150,1.398758,0.836575,0.697885,0.084191,0.912771,0.448495,0.090896,-0.172347,0.780142,...,5.307279e-31,-4.278889e-15,20.0,-0.851370,2.775558e-17,5.551115e-17,2.775558e-17,1.387779e-17,1.734723e-17,0.0


In [11]:
train_X.isnull().sum()
# check_nan_in_df = train_X.isnull()
# print (check_nan_in_df)
train_X.isnull().sum().sum()

282

In [12]:
# train_X.info

In [13]:
class_train_y, class_test_y, class_train_prob_y, class_test_prob_y = random_forest(
            train_X, train_y, test_X, gridsearch=True, print_model_details=True, save_model=True)
performance_training_rf_final = eval.f1(train_y, class_train_y)
performance_test_rf_final = eval.f1(test_y, class_test_y)
confusionmatrix_rf_final = eval.confusion_matrix(test_y, class_test_y, ['check'])
print(performance_test_rf_final) #test performance is reasonable!
print(confusionmatrix_rf_final)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values